# Ваша задача - написать эффективную имплементацию операцию `padded_moe_permute`.
## (Best speed and quality results of 137 people)
Ваша функция должна называться `submission` и иметь следующую сигнатуру:

```
def submission(
    x: torch.Tensor,  # (num_tokens, hidden_size) - входной тензор токенов, каждый размерности hidden_size
    top_experts: torch.Tensor,  # (num_tokens, topk) - для каждого токена указано topk экспертов, которые он активирует
    tokens_per_expert: torch.Tensor, # (num_experts,) - тензор размерности числа экспертов, i-ый элемент - сколько токенов приходит в i-ого эксперта
    topk: int,  # сколько экспертов активируются на каждый токен, например, 8
    num_experts: int,  # сколько всего экспертов в MoE, например, 128
) -> tuple[
    torch.Tensor,  # (max_padded, hidden_size) - padded_tokens, результат пермьюта с паддингами
    torch.Tensor. # (num_experts,) - padded_tokens_per_expert, сколько токенов приходят в каждого эксперта вместе с паддингами
]
```

## Для начала рассмотрим стандартную функцию moe_permute без учета паддингов:

На вход permute-функции приходит тензор размерности (`num_tokens`, `hidden_size`). Обычно в MoE пермьют переставляет токены так, чтобы токены, попадающие в одного эксперта, находились друг за другом.
Например, путь на вход подается
```
x = tensor([[-0.0236, -0.5368, -0.5663],
            [ 0.7778, -0.8583, -0.1123],
            [ 0.1981, -0.3514, -0.9443],
            [-2.0655, -0.9424,  0.9870]])
top_experts = tensor([[1, 3],  # токен 0 выбирает экспертов 1 и 3
                    [2, 5],    # токен 1 выбирает 2 и 5
                    [3, 5],    # токен 2 выбирает 3 и 5
                    [2, 4]])   # токен 3 выбирает 2 и 4
```
В данном случае `topk=2`, каждый токен выбирает 2 экспертов.
Выходной тензор будет иметь размерность `(num_tokens * topk, hidden_size)`, там сначала будут записаны токены для 0ого эксперта, потом для 1ого, потом для 2ого и так далее.
В данном случае:
```
out = tensor([[-0.0236, -0.5368, -0.5663],# токен 0 -> эксперт 1
            [ 0.7778, -0.8583, -0.1123],  # токен 1 -> эксперт 2
            [-2.0655, -0.9424,  0.9870],  # токен 3 -> эксперт 2
            [-0.0236, -0.5368, -0.5663],  # токен 0 -> эксперт 3
            [ 0.1981, -0.3514, -0.9443],  # токен 2 -> эксперт 3
            [-2.0655, -0.9424,  0.9870],  # токен 3 -> эксперт 4
            [ 0.7778, -0.8583, -0.1123],  # токен 1 -> эксперт 5
            [ 0.1981, -0.3514, -0.9443]]) # токен 2 -> эксперт 5
```

**Тензор размерности (num_experts,), который показывает, сколько токенов идут в каждого эксперта, назовем батч сайзами.** В примере выше батч сайзы - это `[1, 2, 2, 1, 2]`.


## Теперь к нашей задаче

При использовании FP8-умножения из `DeepGEMM` (и других современных кернелов) часто ожидается `TMA`-алаймент тензора, то есть появляется требование делимости размерностей на 128.
Это нужно для использования `Tensor Memory Accelerator`-а на H100 для асинхронного копирования из памяти.

В случае `moe_permute` это означает необходимость делимости батч сайзов на 128, то есть чтобы в каждого эксперта приходило делящееся на 128 число токенов.
В примере выше батч сайзы `[1, 2, 2, 1, 2]` станут `[128, 128, 128, 128, 128]`. А, например, `[128, 1, 129]` перейдет в `[128, 128, 256]`.

Чтобы добиться такой гарантии, нам придется западдить результат пермьюта. Теперь он не обязательно будет иметь размерность `num_tokens * topk`, а может содержать дополнительные нулевые токены.
Ваша задача - написать функцию, которая будет делать то же самое, что и обычный `moe_permute`, но уже с паддингами - то есть дополнительно будет гарантировать, что первый токен для каждого эксперта начинается с индекса, делящегося на 128.

В случае входа из примера выше на выходе мы должны получить
```
tensor([[-0.0236, -0.5368, -0.5663], # токен 0 -> эксперт 1
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        ...,
        [ 0.7778, -0.8583, -0.1123],  # токен 1 -> эксперт 2, индекс 128
        [-2.0655, -0.9424,  0.9870],  # токен 3 -> эксперт 2, индекс 129
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]
        ...,
        [ 0.0000,  0.0000,  0.0000]])
```

Для имплементация можно использовать как `Triton`, так и обычный `Torch`.

Имплементация будет проверяться на корректность и производительность. Для прохождения теста на корректность результат вашей функций должен совпадать на `torch.allclose` с выходом eager-имплементации.
Для прохождения теста на производительность ваша функция должна выдавать скорость примерно совпадающую с нашей референсной имлпементацией. Наша референсная имплементация не очень эффективная, поэтому не спешите сразу начинать с `Triton`.

Для простоты вам также дан неэффективный код с `for`-ами.

## Примечание
Логи тестирования можно посмотреть, скачав вывод в тесте 1 на сайте контеста.
Не переименовывайте файл `solution.py`. Ваше решение должно быть в этом файле.

In [1]:
!pip install torch
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import triton
import torch

def pytorch_permute_index_map(tokens, indices):
    if indices.dim() == 1:
        topk = 1
    else:
        topk = indices.size(1)
    flatten_indices = indices.view(-1)
    sorted_indices = torch.argsort(flatten_indices, stable=True)
    num_out_tokens = flatten_indices.size(0)
    permuted_tokens = tokens.index_select(0, sorted_indices[:num_out_tokens] // topk)
    return permuted_tokens, sorted_indices


def torch_basic(x: torch.Tensor, top_experts: torch.Tensor, tokens_per_expert: torch.Tensor, topk: int, num_experts: int):
    block_size = 128
    device = x.device
    num_tokens, hidden_dim = x.shape

    expert_ids_flat = top_experts.view(-1)

    padded_tokens_per_expert = (
        ((tokens_per_expert + block_size - 1) // block_size) * block_size
    ).to(torch.int32)
    padded_offsets = torch.cat([
        torch.zeros(1, dtype=torch.int32, device=device),
        padded_tokens_per_expert.cumsum(dim=0)
    ])
    expert_ids_cpu = expert_ids_flat.cpu().tolist()
    padded_offsets_cpu = padded_offsets.cpu().tolist()

    max_padded_tokens = padded_offsets_cpu[-1]
    padded_tokens = torch.zeros(
        (max_padded_tokens, hidden_dim),
        dtype=x.dtype,
        device=device,
    )

    assignment_groups = [[] for _ in range(num_experts)]
    num_assignments = topk * num_tokens
    for i in range(num_assignments):
        expert_id = expert_ids_cpu[i]
        assignment_groups[expert_id].append(i)

    for e in range(num_experts):
        offset = padded_offsets[e]
        for local_idx, i in enumerate(assignment_groups[e]):
            original_token_idx = i // topk
            token_data = x[original_token_idx]
            target_row = offset + local_idx
            padded_tokens[target_row, :] = token_data

    return padded_tokens, padded_tokens_per_expert

In [3]:
def submission(
    x: torch.Tensor,  # (num_tokens, hidden_size) - входной тензор токенов, каждый размерности hidden_size
    top_experts: torch.Tensor,  # (num_tokens, topk) - для каждого токена указано topk экспертов, которые он активирует
    tokens_per_expert: torch.Tensor, # (num_experts,) - тензор размерности числа экспертов, i-ый элемент - сколько токенов приходит в i-ого эксперта
    topk: int,  # сколько экспертов активируются на каждый токен, например, 8
    num_experts: int,  # сколько всего экспертов в MoE, например, 128
) -> tuple[
    torch.Tensor,  # (max_padded, hidden_size) - padded_tokens, результат пермьюта с паддингами
    torch.Tensor # (num_experts,) - padded_tokens_per_expert, сколько токенов приходят в каждого эксперта вместе с паддингами
]:
    device = x.device
    block_size = 128
    num_tokens, hidden_dim = x.shape

    # Приводим счётчики токенов на эксперта к int64 на нужном девайсе
    tokens_per_expert_long = tokens_per_expert.to(device=device, dtype=torch.long)

    # Паддинги до кратности 128 — как в torch_basic, но в int64
    padded_tokens_per_expert_long = (
        (tokens_per_expert_long + (block_size - 1)) // block_size
    ) * block_size

    # Eager-permute без паддингов: токены уже отсортированы по эксперту
    # permuted_tokens имеет длину sum(tokens_per_expert)
    permuted_tokens, sorted_indices = pytorch_permute_index_map(x, top_experts)
    M = permuted_tokens.size(0)

    # Если по какой-то причине назначений нет
    if M == 0:
        total_padded = int(padded_tokens_per_expert_long.sum().item())
        padded_tokens = torch.zeros(
            (total_padded, hidden_dim), dtype=x.dtype, device=device
        )
        return padded_tokens, padded_tokens_per_expert_long.to(torch.int32)

    # Для каждого assignment (строки permuted_tokens) нужен его expert_id
    # sorted_indices — индексы assignments после сортировки
    flatten_expert_ids = top_experts.view(-1).to(device=device, dtype=torch.long)
    sorted_expert_ids = flatten_expert_ids.index_select(0, sorted_indices)

    # Непаддинговые смещения: где начинается блок каждого эксперта
    # start_unpadded[e] = сумма tokens_per_expert[k] по k < e
    if num_experts > 0:
        tpe_cumsum = tokens_per_expert_long.cumsum(dim=0)
        unpadded_offsets = tpe_cumsum - tokens_per_expert_long
    else:
        unpadded_offsets = torch.zeros(0, dtype=torch.long, device=device)

    # Паддинговые смещения: где начинается блок каждого эксперта в padded-тензоре
    if num_experts > 0:
        padded_cumsum = padded_tokens_per_expert_long.cumsum(dim=0)
        zero = torch.zeros(1, dtype=torch.long, device=device)
        padded_offsets_full = torch.cat([zero, padded_cumsum])  # длина = num_experts + 1
        padded_offsets_start = padded_offsets_full[:-1]         # длина = num_experts
        total_padded = int(padded_offsets_full[-1].item())
    else:
        padded_offsets_start = torch.zeros(0, dtype=torch.long, device=device)
        total_padded = 0

    # Инициализируем выходной тензор нулями (паддинги)
    padded_tokens = torch.zeros(
        (total_padded, hidden_dim), dtype=x.dtype, device=device
    )

    if num_experts > 0:
        # Глобальный индекс assignment'а в permuted_tokens: 0..M-1
        global_idx = torch.arange(M, device=device, dtype=torch.long)

        # Для каждого assignment узнаём, с какого индекса начинается его эксперт
        start_unpadded_for_assign = unpadded_offsets.index_select(0, sorted_expert_ids)
        # Позиция этого assignment внутри блока эксперта (0,1,2,...)
        pos_in_group = global_idx - start_unpadded_for_assign

        # Стартовый индекс блока эксперта в padded-тензоре
        start_padded_for_assign = padded_offsets_start.index_select(0, sorted_expert_ids)

        # Итоговый индекс строки, куда писать этот токен
        dst_indices = start_padded_for_assign + pos_in_group  # shape (M,)

        # Векторизованный scatter: заполняем только реальные токены,
        # остальное остаётся нулями (паддинги)
        padded_tokens.index_copy_(0, dst_indices, permuted_tokens)

    # Возвращаем padded-байтсайзы в int32, как в torch_basic
    return padded_tokens, padded_tokens_per_expert_long.to(torch.int32)

In [4]:
import time
import math
import torch

torch.manual_seed(0)

def make_case(num_experts, topk, hidden_size, num_tokens, device):
    """
    - x: (num_tokens, hidden_size)
    - top_experts: (num_tokens, topk), значения 0..num_experts-1
    - tokens_per_expert: bincount по top_experts
    """
    x = torch.randn(num_tokens, hidden_size, device=device)
    top_experts = torch.randint(
        low=0,
        high=num_experts,
        size=(num_tokens, topk),
        device=device,
    )
    tokens_per_expert = torch.bincount(
        top_experts.reshape(-1),
        minlength=num_experts,
    )
    return x, top_experts, tokens_per_expert


def check_correctness_once(num_experts, topk, hidden_size, num_tokens, device):
    x, top_experts, tpe = make_case(num_experts, topk, hidden_size, num_tokens, device)

    ref_out, ref_padded = torch_basic(x, top_experts, tpe, topk, num_experts)
    my_out, my_padded = submission(x, top_experts, tpe, topk, num_experts)

    if not torch.equal(ref_padded, my_padded.to(ref_padded.dtype)):
        print(f"[FAIL] padded_tokens_per_expert mismatch "
              f"(experts={num_experts}, topk={topk}, hidden={hidden_size}, tokens={num_tokens})")
        print("ref:", ref_padded)
        print("my :", my_padded)
        return False

    if not torch.allclose(ref_out, my_out, atol=1e-6, rtol=1e-5):
        diff = (ref_out - my_out).abs().max().item()
        print(f"[FAIL] output mismatch (max diff={diff:.3e}) "
              f"(experts={num_experts}, topk={topk}, hidden={hidden_size}, tokens={num_tokens})")
        return False

    print(f"[OK]   correctness (experts={num_experts}, topk={topk}, "
          f"hidden={hidden_size}, tokens={num_tokens})")
    return True


def bench_pair(fn_ref, fn_my, warmup=10, iters=50, device="cuda"):
    """
    (t_ref, t_my) — среднее время одного вызова в секундах.
    """
    if device == "cuda":
        torch.cuda.synchronize()

    for _ in range(warmup):
        fn_ref()
        fn_my()
    if device == "cuda":
        torch.cuda.synchronize()

    def run_many(fn):
        if device == "cuda":
            torch.cuda.synchronize()
        t0 = time.perf_counter()
        for _ in range(iters):
            fn()
        if device == "cuda":
            torch.cuda.synchronize()
        t1 = time.perf_counter()
        return (t1 - t0) / iters

    t_ref = run_many(fn_ref)
    t_my = run_many(fn_my)
    return t_ref, t_my


def bench_single_case(num_experts, topk, hidden_size, num_tokens, device):
    print(f"\n=== bench experts={num_experts}, topk={topk}, "
          f"hidden={hidden_size}, tokens={num_tokens} ===")
    x, top_experts, tpe = make_case(num_experts, topk, hidden_size, num_tokens, device)

    ref_out, ref_padded = torch_basic(x, top_experts, tpe, topk, num_experts)
    my_out, my_padded = submission(x, top_experts, tpe, topk, num_experts)
    assert torch.equal(ref_padded, my_padded.to(ref_padded.dtype))
    assert torch.allclose(ref_out, my_out, atol=1e-6, rtol=1e-5)

    def call_ref():
        torch_basic(x, top_experts, tpe, topk, num_experts)

    def call_my():
        submission(x, top_experts, tpe, topk, num_experts)

    t_ref, t_my = bench_pair(call_ref, call_my, device=device)
    ratio = t_my / t_ref if t_ref > 0 else float("inf")

    print(f"ref: {t_ref * 1e3:.3f} ms  my: {t_my * 1e3:.3f} ms  "
          f"ratio={ratio * 100:.1f}%")

    return t_ref, t_my


def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # 1) sanity-check
    print("\n>>> correctness quick check")
    for num_experts in [16, 128]:
        for topk in [1, 4, 8]:
            for hidden in [256, 1024]:
                for num_tokens in [128, 512]:
                    ok = check_correctness_once(num_experts, topk, hidden, num_tokens, device)
                    if not ok:
                        print("Корректность сломана, дальше бенчить нет смысла.")
                        return

    # 2) `test_speed_vs_eager[16-4-256-128]`
    print("\n>>> critical case (по логам тестов: 16-4-256-128)")
    bench_single_case(
        num_experts=16,
        topk=4,
        hidden_size=256,
        num_tokens=128,
        device=device,
    )

    # 3) [16/128]-[1/4/8]-[256/2048/8192]-[128/1024]
    configs = []
    for num_experts in [16, 128]:
        for topk in [1, 4, 8]:
            for hidden in [256, 2048, 8192]:
                for num_tokens in [128, 1024]:
                    configs.append((num_experts, topk, hidden, num_tokens))

    print("\n>>> grid benchmark (примерно как в тестах):")
    for (ne, tk, h, nt) in configs:
        bench_single_case(ne, tk, h, nt, device=device)


if __name__ == "__main__":
    main()

Device: cuda

>>> correctness quick check
[OK]   correctness (experts=16, topk=1, hidden=256, tokens=128)
[OK]   correctness (experts=16, topk=1, hidden=256, tokens=512)
[OK]   correctness (experts=16, topk=1, hidden=1024, tokens=128)
[OK]   correctness (experts=16, topk=1, hidden=1024, tokens=512)
[OK]   correctness (experts=16, topk=4, hidden=256, tokens=128)
[OK]   correctness (experts=16, topk=4, hidden=256, tokens=512)
[OK]   correctness (experts=16, topk=4, hidden=1024, tokens=128)
[OK]   correctness (experts=16, topk=4, hidden=1024, tokens=512)
[OK]   correctness (experts=16, topk=8, hidden=256, tokens=128)
[OK]   correctness (experts=16, topk=8, hidden=256, tokens=512)
[OK]   correctness (experts=16, topk=8, hidden=1024, tokens=128)
[OK]   correctness (experts=16, topk=8, hidden=1024, tokens=512)
[OK]   correctness (experts=128, topk=1, hidden=256, tokens=128)
[OK]   correctness (experts=128, topk=1, hidden=256, tokens=512)
[OK]   correctness (experts=128, topk=1, hidden=1024, 

In [5]:
# LOGS / output

'''
============================= test session starts ==============================
platform linux -- Python 3.11.14, pytest-9.0.0, pluggy-1.6.0 -- /opt/conda/bin/python3.11
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: /workspace
plugins: hypothesis-6.141.0
collecting ... collected 72 items

tests.py::test_quality[16-1-256-128] PASSED                              [  1%]
tests.py::test_quality[16-1-256-1024] PASSED                             [  2%]
tests.py::test_quality[16-1-2048-128] PASSED                             [  4%]
tests.py::test_quality[16-1-2048-1024] PASSED                            [  5%]
tests.py::test_quality[16-1-8192-128] PASSED                             [  6%]
tests.py::test_quality[16-1-8192-1024] PASSED                            [  8%]
tests.py::test_quality[16-4-256-128] PASSED                              [  9%]
tests.py::test_quality[16-4-256-1024] PASSED                             [ 11%]
tests.py::test_quality[16-4-2048-128] PASSED                             [ 12%]
tests.py::test_quality[16-4-2048-1024] PASSED                            [ 13%]
tests.py::test_quality[16-4-8192-128] PASSED                             [ 15%]
tests.py::test_quality[16-4-8192-1024] PASSED                            [ 16%]
tests.py::test_quality[16-8-256-128] PASSED                              [ 18%]
tests.py::test_quality[16-8-256-1024] PASSED                             [ 19%]
tests.py::test_quality[16-8-2048-128] PASSED                             [ 20%]
tests.py::test_quality[16-8-2048-1024] PASSED                            [ 22%]
tests.py::test_quality[16-8-8192-128] PASSED                             [ 23%]
tests.py::test_quality[16-8-8192-1024] PASSED                            [ 25%]
tests.py::test_quality[128-1-256-128] PASSED                             [ 26%]
tests.py::test_quality[128-1-256-1024] PASSED                            [ 27%]
tests.py::test_quality[128-1-2048-128] PASSED                            [ 29%]
tests.py::test_quality[128-1-2048-1024] PASSED                           [ 30%]
tests.py::test_quality[128-1-8192-128] PASSED                            [ 31%]
tests.py::test_quality[128-1-8192-1024] PASSED                           [ 33%]
tests.py::test_quality[128-4-256-128] PASSED                             [ 34%]
tests.py::test_quality[128-4-256-1024] PASSED                            [ 36%]
tests.py::test_quality[128-4-2048-128] PASSED                            [ 37%]
tests.py::test_quality[128-4-2048-1024] PASSED                           [ 38%]
tests.py::test_quality[128-4-8192-128] PASSED                            [ 40%]
tests.py::test_quality[128-4-8192-1024] PASSED                           [ 41%]
tests.py::test_quality[128-8-256-128] PASSED                             [ 43%]
tests.py::test_quality[128-8-256-1024] PASSED                            [ 44%]
tests.py::test_quality[128-8-2048-128] PASSED                            [ 45%]
tests.py::test_quality[128-8-2048-1024] PASSED                           [ 47%]
tests.py::test_quality[128-8-8192-128] PASSED                            [ 48%]
tests.py::test_quality[128-8-8192-1024] PASSED                           [ 50%]
tests.py::test_speed_vs_eager[16-4-256-128] PASSED                       [ 51%]
tests.py::test_speed_vs_eager[16-4-256-1024] PASSED                      [ 52%]
tests.py::test_speed_vs_eager[16-4-256-16384] PASSED                     [ 54%]
tests.py::test_speed_vs_eager[16-4-2048-128] PASSED                      [ 55%]
tests.py::test_speed_vs_eager[16-4-2048-1024] PASSED                     [ 56%]
tests.py::test_speed_vs_eager[16-4-2048-16384] PASSED                    [ 58%]
tests.py::test_speed_vs_eager[16-4-8192-128] PASSED                      [ 59%]
tests.py::test_speed_vs_eager[16-4-8192-1024] PASSED                     [ 61%]
tests.py::test_speed_vs_eager[16-4-8192-16384] PASSED                    [ 62%]
tests.py::test_speed_vs_eager[16-8-256-128] PASSED                       [ 63%]
tests.py::test_speed_vs_eager[16-8-256-1024] PASSED                      [ 65%]
tests.py::test_speed_vs_eager[16-8-256-16384] PASSED                     [ 66%]
tests.py::test_speed_vs_eager[16-8-2048-128] PASSED                      [ 68%]
tests.py::test_speed_vs_eager[16-8-2048-1024] PASSED                     [ 69%]
tests.py::test_speed_vs_eager[16-8-2048-16384] PASSED                    [ 70%]
tests.py::test_speed_vs_eager[16-8-8192-128] PASSED                      [ 72%]
tests.py::test_speed_vs_eager[16-8-8192-1024] PASSED                     [ 73%]
tests.py::test_speed_vs_eager[16-8-8192-16384] PASSED                    [ 75%]
tests.py::test_speed_vs_eager[128-4-256-128] PASSED                      [ 76%]
tests.py::test_speed_vs_eager[128-4-256-1024] PASSED                     [ 77%]
tests.py::test_speed_vs_eager[128-4-256-16384] PASSED                    [ 79%]
tests.py::test_speed_vs_eager[128-4-2048-128] PASSED                     [ 80%]
tests.py::test_speed_vs_eager[128-4-2048-1024] PASSED                    [ 81%]
tests.py::test_speed_vs_eager[128-4-2048-16384] PASSED                   [ 83%]
tests.py::test_speed_vs_eager[128-4-8192-128] PASSED                     [ 84%]
tests.py::test_speed_vs_eager[128-4-8192-1024] PASSED                    [ 86%]
tests.py::test_speed_vs_eager[128-4-8192-16384] PASSED                   [ 87%]
tests.py::test_speed_vs_eager[128-8-256-128] PASSED                      [ 88%]
tests.py::test_speed_vs_eager[128-8-256-1024] PASSED                     [ 90%]
tests.py::test_speed_vs_eager[128-8-256-16384] PASSED                    [ 91%]
tests.py::test_speed_vs_eager[128-8-2048-128] PASSED                     [ 93%]
tests.py::test_speed_vs_eager[128-8-2048-1024] PASSED                    [ 94%]
tests.py::test_speed_vs_eager[128-8-2048-16384] PASSED                   [ 95%]
tests.py::test_speed_vs_eager[128-8-8192-128] PASSED                     [ 97%]
tests.py::test_speed_vs_eager[128-8-8192-1024] PASSED                    [ 98%]
tests.py::test_speed_vs_eager[128-8-8192-16384] PASSED                   [100%]

============================= 72 passed in 49.09s ==============================
========= best ::test_quality and ::test_speed_vs_eager submission yet =========
============================= test session ends ================================
'''

"\n============================= test session starts ==============================\nplatform linux -- Python 3.11.14, pytest-9.0.0, pluggy-1.6.0 -- /opt/conda/bin/python3.11\ncachedir: .pytest_cache\nhypothesis profile 'default'\nrootdir: /workspace\nplugins: hypothesis-6.141.0\ncollecting ... collected 72 items\n\ntests.py::test_quality[16-1-256-128] PASSED                              [  1%]\ntests.py::test_quality[16-1-256-1024] PASSED                             [  2%]\ntests.py::test_quality[16-1-2048-128] PASSED                             [  4%]\ntests.py::test_quality[16-1-2048-1024] PASSED                            [  5%]\ntests.py::test_quality[16-1-8192-128] PASSED                             [  6%]\ntests.py::test_quality[16-1-8192-1024] PASSED                            [  8%]\ntests.py::test_quality[16-4-256-128] PASSED                              [  9%]\ntests.py::test_quality[16-4-256-1024] PASSED                             [ 11%]\ntests.py::test_quality[16-4-2048-1